In [1]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("Python Spark SQL basic example") \
	    .config("spark.some.config.option", "some-value") \
	    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/19 04:33:43 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/04/19 04:33:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/04/19 04:33:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/04/19 04:33:44 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
initial_df = spark.read.csv(path='Rodent_Inspection.csv',header=True)

In [3]:
initial_df.count()

2354285

In [13]:
#Get All attribute names and it's types
for attr in initial_df.dtypes:
    print(attr[0] + ", " + attr[1])

INSPECTION_TYPE, string
JOB_TICKET_OR_WORK_ORDER_ID, string
JOB_ID, string
JOB_PROGRESS, string
BBL, string
BORO_CODE, string
BLOCK, string
LOT, string
HOUSE_NUMBER, string
STREET_NAME, string
ZIP_CODE, string
X_COORD, string
Y_COORD, string
LATITUDE, string
LONGITUDE, string
BOROUGH, string
INSPECTION_DATE, string
RESULT, string
APPROVED_DATE, string
LOCATION, string
INSP_DATE, timestamp
APP_DATE, timestamp


In [14]:
#cast column indicating date with string type to timestamp
import pyspark.sql.functions as F
initial_df=initial_df.withColumn('INSP_DATE', F.to_timestamp(F.col("INSPECTION_DATE"), "MM/dd/yyyy hh:mm:ss a")) \
                     .withColumn('APP_DATE', F.to_timestamp(F.col("APPROVED_DATE"), "MM/dd/yyyy hh:mm:ss a"))
initial_df.show(truncate=False)

+---------------+---------------------------+---------+------------+----------+---------+-----+----+------------+-------------+--------+-------+-------+--------+---------+-------------+----------------------+------------------+----------------------+----------+-------------------+-------------------+
|INSPECTION_TYPE|JOB_TICKET_OR_WORK_ORDER_ID|JOB_ID   |JOB_PROGRESS|BBL       |BORO_CODE|BLOCK|LOT |HOUSE_NUMBER|STREET_NAME  |ZIP_CODE|X_COORD|Y_COORD|LATITUDE|LONGITUDE|BOROUGH      |INSPECTION_DATE       |RESULT            |APPROVED_DATE         |LOCATION  |INSP_DATE          |APP_DATE           |
+---------------+---------------------------+---------+------------+----------+---------+-----+----+------------+-------------+--------+-------+-------+--------+---------+-------------+----------------------+------------------+----------------------+----------+-------------------+-------------------+
|Initial        |13407377                   |PC7882384|1           |3019260010|3        |01926

In [15]:
initial_df.createOrReplaceTempView("rodent_inspection")
spark.sql("SELECT min(INSPECTION_DATE), max(INSPECTION_DATE) FROM rodent_inspection").show()

+--------------------+--------------------+
|min(INSPECTION_DATE)|max(INSPECTION_DATE)|
+--------------------+--------------------+
|01/01/2001 03:15:...|12/31/2022 12:50:...|
+--------------------+--------------------+



In [16]:
#check strange date
spark.sql("SELECT min(INSP_DATE), max(INSP_DATE) FROM rodent_inspection").show()

+-------------------+-------------------+
|     min(INSP_DATE)|     max(INSP_DATE)|
+-------------------+-------------------+
|1918-10-19 14:34:44|2029-01-01 09:35:51|
+-------------------+-------------------+



In [17]:
spark.sql("SELECT INSP_DATE, INSPECTION_DATE, APP_DATE, APPROVED_DATE FROM rodent_inspection where year(INSP_DATE) < 2001").show()

+-------------------+--------------------+-------------------+--------------------+
|          INSP_DATE|     INSPECTION_DATE|           APP_DATE|       APPROVED_DATE|
+-------------------+--------------------+-------------------+--------------------+
|1935-02-22 10:19:31|02/22/1935 10:19:...|2017-02-28 08:52:04|02/28/2017 08:52:...|
|1956-09-12 10:56:28|09/12/1956 10:56:...|2019-09-16 16:23:22|09/16/2019 04:23:...|
|2000-10-10 09:45:35|10/10/2000 09:45:...|2017-10-17 10:42:19|10/17/2017 10:42:...|
|1950-02-13 10:50:01|02/13/1950 10:50:...|2018-02-20 16:15:58|02/20/2018 04:15:...|
|1918-10-19 14:34:44|10/19/1918 02:34:...|2018-10-26 08:59:20|10/26/2018 08:59:...|
|2000-12-31 11:50:57|12/31/2000 11:50:...|2018-07-20 09:09:30|07/20/2018 09:09:...|
|1958-03-08 12:58:04|03/08/1958 12:58:...|2019-03-13 13:31:28|03/13/2019 01:31:...|
|1930-04-14 12:57:46|04/14/1930 12:57:...|2014-04-15 09:27:03|04/15/2014 09:27:...|
|1971-05-03 08:18:12|05/03/1971 08:18:...|2013-05-08 17:04:32|05/08/2013 05:

In [18]:
spark.sql("SELECT JOB_TICKET_OR_WORK_ORDER_ID,INSP_DATE, INSPECTION_DATE, APP_DATE, APPROVED_DATE FROM rodent_inspection where year(INSP_DATE) >2023").show()

+---------------------------+-------------------+--------------------+-------------------+--------------------+
|JOB_TICKET_OR_WORK_ORDER_ID|          INSP_DATE|     INSPECTION_DATE|           APP_DATE|       APPROVED_DATE|
+---------------------------+-------------------+--------------------+-------------------+--------------------+
|                    2562758|2025-03-26 14:25:19|03/26/2025 02:25:...|2016-03-31 12:15:55|03/31/2016 12:15:...|
|                    2569167|2024-05-24 01:12:50|05/24/2024 01:12:...|2016-05-26 08:44:50|05/26/2016 08:44:...|
|                    2555783|2029-01-01 09:35:51|01/01/2029 09:35:...|2016-02-01 11:11:16|02/01/2016 11:11:...|
+---------------------------+-------------------+--------------------+-------------------+--------------------+



In [19]:

spark.sql("SELECT count(*)  FROM rodent_inspection where LOCATION is not null ").show()

+--------+
|count(1)|
+--------+
| 2340191|
+--------+



In [20]:
spark.sql("SELECT count(*)  FROM rodent_inspection where LATITUDE is not null").show()

+--------+
|count(1)|
+--------+
| 2341308|
+--------+



In [21]:
spark.sql("SELECT count(*)  FROM rodent_inspection where LONGITUDE is not null").show()

+--------+
|count(1)|
+--------+
| 2341308|
+--------+



In [25]:
spark.sql("SELECT count(*), year(INSP_DATE)   FROM rodent_inspection where LONGITUDE is not null and LONGITUDE is not null and year(INSP_DATE) between 2001 and 2023 group by year(INSP_DATE) order by year(INSP_DATE)").show()

+--------+-----------------------------+
|count(1)|year(CAST(INSP_DATE AS DATE))|
+--------+-----------------------------+
|      14|                         2001|
|       5|                         2002|
|       5|                         2003|
|       1|                         2007|
|       5|                         2008|
|   19496|                         2009|
|  152056|                         2010|
|  157472|                         2011|
|  172964|                         2012|
|  132444|                         2013|
|  134785|                         2014|
|  135819|                         2015|
|  181306|                         2016|
|  249012|                         2017|
|  254821|                         2018|
|  248287|                         2019|
|   72605|                         2020|
|  107583|                         2021|
|  244588|                         2022|
|   78007|                         2023|
+--------+-----------------------------+

